![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# Transferencia de aprendizaje simple con `pyTorch`

## Aprendizaje Automático Aplicado

### Maestría en Ciencia de Datos

#### **Julio Waissman**, 2024

[**Abrir en google Colab**](https://colab.research.google.com/github/mcd-unison/aaa-curso/blob/main/ejemplos/transfer_pytorch.ipynb)


*Tomado de [este tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)*


En la practica casi nadie inicia aprendiendo un modelo desde cero, ya que lo normal no es contar con un conjunto de datos de entrenamiento lo suficientemente grande o contar con la infraestructura necesaria para entrenar dichos modelos. Para más información sobre transferencia del aprendizaje puedes ver [esta presentacioncita](https://github.com/mcd-unison/aaa-curso/raw/main/slides/transfer_learning.pdf)

En su lugar, lo normal es descargar algún modelo preentrenado (hay unos muy famosos), la mayoría entrenados con el conjunto de imágenes [ImageNet](https://www.image-net.org). Los dos escenarios que vamos a ver en esta libreta y son los más comunes para transferencia del aprendizaje son:

1. **Ajuste fino (finetuning)**. El lugar de inicializar en forma aleatorio, usamos una red preentrenada como inicializador, y luego entrenamos todos los parámetros, aunque con una tasa de aprendizaje muy pequeña.

2. **CNN preentrenada como un generador de características**. Vamos a congelar todos los pesas de todas las capas de una red preentrenada, y vamos a sustitur la última capa. Así, nuestra red en realidad es un perceptron, y el resto es un modelo fijo de extracción de caracteríaticas. Esto claramente se puede hacer con varias capas.

In [2]:
#Las librarias que se necesitan

# Las de siempre
import os
import time
import numpy as np
import matplotlib.pyplot as plt
plt.ion()   # interactive mode


# Torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

# Torch para vision (con modelos)
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision import models

# Para manipular imágenes
from PIL import Image

# Para el aprendizaje
from tempfile import TemporaryDirectory

# Usamos GPU (poner un entorno de jecución adecuado)
cudnn.benchmark = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Cargando datos

Vamos a usar el conjunto

In [16]:
# Descargamos los datos
%%capture
!curl -O https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip -y hymenoptera_data.zip

In [18]:
# Aumento de imágenes y normalización para entrenamiento
# Normalización para clasificación

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# Generamos los dataset de entrenamiento y prueba
# a partir de datos que tenemos en archivos
data_dir = 'hymenoptera_data'
image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x),
        data_transforms[x]
    )
    for x in ['train', 'val']
}

# Generamos los DataLoaders
# a partir de los dataset
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x],
        batch_size=4,
        shuffle=True,
        num_workers=4
    )
    for x in ['train', 'val']
}

# Tamaño de cada conjunto de datos
dataset_sizes = {
    x: len(image_datasets[x]) for x in ['train', 'val']
}

# Nombres de las clases (un problema de clasificación binaria)
class_names = image_datasets['train'].classes

Vamos a ver algunas de las imágenes para ilustrar el *Image augmentation* y como funciona.

In [32]:
def imshow(inp, title=None):

    # Ordena los ejes como pide plt.imshow
    inp = inp.numpy().transpose((1, 2, 0))

    # Desnormaliza la imagen normalizada
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    # Despliega la imagen concatenada
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# Un minibatch
inputs, classes = next(iter(dataloaders['train']))

# Un grid para visualizar las imágenes
out = torchvision.utils.make_grid(inputs)

#Mostrando las imágenes
imshow(out, title=[class_names[x] for x in classes])

## 2. Funciones de entrenamiento

Vamos a poner la función de entrenamiento que vamos a llamar. Esto se puede reutilizar en otros problenas con `pyTorch`, y es un poco demasiado a pié si ya estás acostumbrado a usar [Keras](https://keras.io/).

Vamos a poner una funcionzota que haga todo el aprendizaje y otra para visualizar las predicciones una vez que el modelo esté entrenado.

In [41]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    """
    Entrenamiento de la red, con entrenamiento y validación

    """
    since = time.time()

    # Directorio temporal para guardar checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Cada epoch va a terne un paso de entrenamiento y uno de validación
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                # Por cada minibatch (de entrenamiento y luego de validación)
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # gradientes a zero primero
                    optimizer.zero_grad()

                    # Solo calculamos gradientes en entrenamiento
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # Si entrenamiento paso de optimización cada minibatch
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                # Al final del epoch completo,
                # usa un método de degeneración de peso (scheduler)
                if phase == 'train':
                    scheduler.step()

                # statistics
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # Si el modelo es mejor al del checkpoint, actualiza el checkpoint
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [42]:
def visualize_model(model, num_images=6):
    """
    Hace la preducción de num_images del conjunto de validación
    y lo despliega para ver como realiza la clasificación

    La predicción la hace en gpu, pero la vosualización
    la pasa a cpu para ahorrar recursos

    """
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## 3. Ejemplo de ajuste fino

Vamos a utilizar un modelo y un conjunto de pesos preentrenados disponibles con [`torchvision`](https://pytorch.org/vision/stable/index.html). Los modelos disponibles (por default), se pueden encontrar [en este enlace](https://pytorch.org/vision/stable/models.html).

Para este ejemplo vamos a usar un modelo relativamente simple, [ResNet18](https://arxiv.org/abs/1512.03385), y los datos preentrenados son [`IMAGENET1K_V1`](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet18.html#torchvision.models.ResNet18_Weights). Se va a cargar entonces un archivo con:

- 44.7 MB de tamaño
- 11,689,512 parámetros entrenados

Vamos a cargar el modelo y establecer el criterio (función de pérdida) optimizador y calendarizador (para la degeneración de la tasa de aprendizaje)

In [ ]:
# Usamos resnet18
model_ft = models.resnet18(weights='IMAGENET1K_V1')

# Obtenemos cuantas entradas tiene la capa final
num_ftrs = model_ft.fc.in_features

# Sustituimos la capa final por un clasificador binario
model_ft.fc = nn.Linear(num_ftrs, 2)

# Mandamos el modelo al GPU
model_ft = model_ft.to(device)

# La funcion de pérdida clásica (con logits)
criterion = nn.CrossEntropyLoss()

# Todos los parámetros se van a optimizar
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Degeneración de la tasa de aprendizaje en 0.1 cada 7 epoch
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Y ahora si a entrenar. Toma de 15 a 25 minutos en CPU y menos de un minuto en GPU aproximadamente y dependiendo del humor de `colab`

In [ ]:
model_ft = train_model(
    model_ft,
    criterion,
    optimizer_ft,
    exp_lr_scheduler,
    num_epochs=25
)

Y vamos a visualizar que tal

In [ ]:
visualize_model(model_ft)

## 4. Ejemplo como generador de características

Aqui vamos a congelar todos los parámetros, salvo los de la última capa (que es la única que cambiamos en la arquitectura, por cierto.

Así que va a ser muy similar al ajuste fino, solamente que vamos a poner como `requires_grad = False`a todos los parámetros que no sean de la última capa.

In [48]:
# Carga el modelo y los parámetros preentrenados
model_conv = torchvision.models.resnet18(weights='IMAGENET1K_V1')

# Pone todos los parámetros como no entrenables
for param in model_conv.parameters():
    param.requires_grad = False

# Cambia la última capa.
# Cuando hay nuevos parámetros, estos son entrenables por default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

# Pasa el modelo al GPU
model_conv = model_conv.to(device)

# Criterio de pérdida
criterion = nn.CrossEntropyLoss()

# Optimizador, solo se ponen los parámetros de la última capa
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Reduce la tasa de aprendizaje en 0.1 cada 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

En CPU tarda menos de la mitad de tiempo que el ajuste de todos los parámetros y los resultados parecen mejores.

In [ ]:
model_conv = train_model(
    model_conv,
    criterion,
    optimizer_conv,
    exp_lr_scheduler,
    num_epochs=25
)

In [ ]:
visualize_model(model_conv)

## 5. Usando el modelo entrenado

Vamos a hacer una funcioncita para poder usarlo con una imagen cualquiera

In [51]:
def visualize_model_predictions(model, img_path):
    was_training = model.training
    model.eval()

    # Vamos a cargar y transformar la imagen
    # Usamos las transformaciones definidas para validación
    img = Image.open(img_path)
    img = data_transforms['val'](img)
    img = img.unsqueeze(0)
    img = img.to(device)

    # Realiza la predicción y despliega el resultado
    with torch.no_grad():
        outputs = model(img)
        _, preds = torch.max(outputs, 1)

        ax = plt.subplot(2,2,1)
        ax.axis('off')
        ax.set_title(f'Predicted: {class_names[preds[0]]}')
        imshow(img.cpu().data[0])

        model.train(mode=was_training)

In [ ]:
visualize_model_predictions(
    model_conv,
    img_path='hymenoptera_data/val/bees/72100438_73de9f17af.jpg'
)